In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker
import os
import seaborn as sns
from scipy import stats
from mlxtend.evaluate import permutation_test

In [ ]:
A=pd.read_pickle("data/allyears-avg_plate.pkl")


In [ ]:
# Permutation test for combined off-festival vs festival period
p_value = permutation_test( A.loc[A['festival'] == 'during', 'Bioindicator'] , 
                            A.loc[A['festival'] != 'during', 'Bioindicator'] , 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

In [ ]:
for c in ['Bioindicator','Coliform']:
    p1 = permutation_test( A.loc[A['year'] == 16 , c] , 
                                A.loc[A['year'] == 17, c] , 
                               method='approximate',
                               num_rounds=10000,
                               seed=0)
    p2 = permutation_test( A.loc[A['year'] == 16 , c] , 
                                A.loc[A['year'] == 20, c] , 
                               method='approximate',
                               num_rounds=10000,
                               seed=0)
    p3 = permutation_test( A.loc[A['year'] == 17 , c] , 
                                A.loc[A['year'] == 20, c] , 
                               method='approximate',
                               num_rounds=10000,
                               seed=0)
    print(f"16vs17 = {p1}, 16vs20 = {p2}, 17vs20 = {p3} on {c}")

In [ ]:
p_value = permutation_test( A.loc[(A['year']==16) & (A['festival'] == 'during'), 'Bioindicator'] , 
                            A.loc[(A['year']==17) & (A['festival'] == 'during'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

In [ ]:
p_value = permutation_test( A.loc[(A['year']==16) & (A['festival'] == 'during'), 'Bioindicator'] , 
                            A.loc[(A['year']==16) & (A['festival'] != 'during'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

In [ ]:
p_value = permutation_test( A.loc[(A['year']==17) & (A['festival'] == 'during'), 'Bioindicator'] , 
                            A.loc[(A['year']==17) & (A['festival'] != 'during'), 'Bioindicator'], 
                           method='approximate',
                           num_rounds=10000,
                           seed=0)
print(p_value)

In [ ]:
main_vars=['Bioindicator', 'Coliform', 'Bioindicator UV+',  'total' ,'rain', 'rooms']
A[main_vars].corr(method ='spearman')

In [ ]:
def append_vars(ds, cols):
    v=ds[cols]
    X=ds.drop(columns=cols)
    return pd.concat([X.assign(cfu=ds[col], name=ds['year'].apply(lambda y: f"{col} {y}")) for col in cols]).dropna()

In [ ]:
A2=append_vars(A, ['Bioindicator', 'Bioindicator UV+', 'Coliform', 'total'])

In [ ]:
A.query('year==16')[main_vars].corr(method ='spearman')

In [ ]:
A.query('year==17')[main_vars].corr(method ='spearman')

In [ ]:
A.query('year==20')[main_vars].corr(method ='spearman')

In [ ]:
A[main_vars].corr(method ='spearman') 

In [ ]:
A2[['cfu', 'rooms', 'rain']].corr(method ='spearman')

In [ ]:
def bootstrap_ci(df, variable, classes, repetitions = 1000, alpha = 0.05, random_state=None): 
    
    df = df[[variable, classes]]
    bootstrap_sample_size = len(df) 
    
    mean_diffs = []
    for i in range(repetitions):
            bootstrap_sample = df.sample(n = bootstrap_sample_size, replace = True, random_state = random_state)
            mean_diff = bootstrap_sample.groupby(classes).mean().iloc[1,0] - bootstrap_sample.groupby(classes).mean().iloc[0,0]
            mean_diffs.append(mean_diff)
    # confidence interval
    left = np.percentile(mean_diffs, alpha/2*100)
    right = np.percentile(mean_diffs, 100-alpha/2*100)
    # point estimate
    point_est = df.groupby(classes).mean().iloc[1,0] - df.groupby(classes).mean().iloc[0,0]
    print('Point estimate of difference between means:', round(point_est,2))
    print((1-alpha)*100,'%','confidence interval for the difference between means:', (round(left,2), round(right,2)))


In [ ]:
import arch.data.frenchdata
from arch.bootstrap import StationaryBootstrap,IndependentSamplesBootstrap


In [ ]:
def foi(X):
    return X[['cfu', 'rooms', 'rain']].corr(method ='spearman')['cfu']['rain']
    
bs = StationaryBootstrap(12, A2)
results = bs.apply(foi, 2500)
SR = pd.DataFrame(results[:, -1:], columns=["SR"])
fig = SR.hist(bins=40)


In [ ]:
ci = bs.conf_int(foi, 1000, method="percentile", reuse=True)
ci


In [ ]:
def mean_diff(x, y):
    return x.mean() - y.mean()

bs = IndependentSamplesBootstrap(A.loc[A['year'] == 16], A.loc[A['year'] == 20])
print(bs.conf_int(mean_diff, method="studentized"))
